# Tables generation

In this short notebook I will show you how to generate different tables from csv files of player stats for each match from season 2017-2018 including top 5 european leagues.

Let's go!

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np
import glob
import seaborn as sns
from tqdm import tqdm

### 1. Load csvs

In [2]:
leagues_path = 'leagues/*/*/players*tillnow.csv'

In [3]:
leagues = glob.glob(leagues_path)

In [4]:
player_match_data = pd.DataFrame()

for csv in tqdm(leagues, desc = 'player_match_dataenating league data into one csv'):

    df = pd.read_csv(csv, thousands = ',')

    player_match_data = pd.concat([player_match_data,df])
        
player_match_data.loc[player_match_data.gameweek.isna(), 'gameweek'] = player_match_data.loc[player_match_data.gameweek.isna(), 'gameweek_x']
player_match_data.loc[player_match_data.gameweek.isna(), 'gameweek'] = player_match_data.loc[player_match_data.gameweek.isna(), 'gameweek_y']

player_match_data.drop(['gameweek_x', 'gameweek_y'], axis = 1, inplace = True)
    
player_match_data["date"] = pd.to_datetime(player_match_data['date'])
player_match_data.loc[player_match_data.attendance.isna(), 'attendance']  =0

player_match_dataenating league data into one csv: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35/35 [00:05<00:00,  6.23it/s]


### 2. Define seasons

In [5]:
date_ranges = [
    ('2017-07-01', '2018-07-01'),
    ('2018-07-01', '2019-07-01'),
    ('2019-07-01', '2020-08-03'),
    ('2020-08-03', '2021-07-01'),
    ('2021-07-01', '2022-07-01'),
    ('2022-07-01', '2023-07-01'),
    ('2023-07-01', '2024-07-01')
]

for start_date, end_date in date_ranges:
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    mask = (player_match_data['date'] >= start_date) & (player_match_data['date'] < end_date)
    player_match_data.loc[mask, 'season'] = f'{start_date.year}_{end_date.year}'  

### 3. Add important columns

In [6]:
player_match_data['starter'] = player_match_data.apply(lambda row: 0 if 
                                                       row['name'].startswith('\xa0\xa0\xa0') else 
                                                       1, axis = 1)
player_match_data.name = player_match_data.name.str.strip()

In [7]:
player_match_data[['home_score', 'away_score']] = player_match_data.apply(
    lambda row: pd.Series([int(x) for x in row['score'].split('–')]), axis = 1)

player_match_data[['ht_home_score', 'ht_away_score']] = player_match_data.apply(
    lambda row: pd.Series([int(x) for x in row['ht_score'].split('–')]), axis = 1)

In [8]:
player_match_data['hda_match_winner'] = player_match_data.apply(lambda row: 'h' if row.home_score>row.away_score else 
                                                          'a' if row.home_score<row.away_score else 
                                                          'd', axis = 1)

player_match_data['ht_hda_match_winner'] = player_match_data.apply(lambda row: 'h' if 
                                                             row.ht_home_score>row.ht_away_score else 
                                                             'a' if row.ht_home_score<row.ht_away_score else 
                                                             'd', axis = 1)


player_match_data['match_home_win'] = (player_match_data['hda_match_winner'] == 'h').astype(int)
player_match_data['match_draw'] = (player_match_data['hda_match_winner'] == 'd').astype(int)
player_match_data['match_win'] = (player_match_data['hda_match_winner'] == 'a').astype(int)

player_match_data['match_home_lose'] = (player_match_data['hda_match_winner'] == 'a').astype(int)
player_match_data['match_away_lose'] = (player_match_data['hda_match_winner'] == 'h').astype(int)

player_match_data['ht_match_home_win'] = (player_match_data['ht_hda_match_winner'] == 'h').astype(int)
player_match_data['ht_match_draw'] = (player_match_data['ht_hda_match_winner'] == 'd').astype(int)
player_match_data['ht_match_away_win'] = (player_match_data['ht_hda_match_winner'] == 'a').astype(int)

player_match_data['ht_match_home_lose'] = (player_match_data['ht_hda_match_winner'] == 'a').astype(int)
player_match_data['ht_match_away_lose'] = (player_match_data['ht_hda_match_winner'] == 'h').astype(int)

player_match_data.drop('hda_match_winner', inplace = True, axis = 1)
player_match_data.drop('ht_hda_match_winner', inplace = True, axis = 1)

In [9]:
player_match_data['opponent'] = player_match_data.apply(lambda row: 
                                                           row.home_team if row.away_team==row.team else 
                                                           row.away_team if row.home_team==row.team else None, axis = 1)

player_match_data['team_winner'] = player_match_data.apply(lambda row: 
                                                           row.home_team if row.home_score>row.away_score else 
                                                           row.away_team if row.home_score<row.away_score else 
                                                           'draw', axis = 1)

player_match_data['ht_team_winner'] = player_match_data.apply(
    lambda row: row.home_team if row.ht_home_score>row.ht_away_score else 
    row.away_team if row.ht_home_score<row.ht_away_score else 'draw', 
                                                              axis = 1)

player_match_data['home_away'] = player_match_data.apply(lambda row: 
                                                           'home' if row.home_team==row.team else 
                                                           'away' if row.away_team==row.team else None, axis = 1)

player_match_data['team_home_win'] = player_match_data.apply(lambda row: 
                                                           1 if row.home_away=='home' and row.team_winner==row.team 
                                                             else 0, axis = 1)

player_match_data['team_home_draw'] = player_match_data.apply(lambda row: 
                                                           1 if row.home_away=='home' and row.team_winner=='draw' 
                                                             else 0, axis = 1)

player_match_data['team_home_lose'] = player_match_data.apply(lambda row: 
                                                           1 if row.home_away=='home' and row.team_winner==row.opponent 
                                                             else 0, axis = 1)

player_match_data['team_away_win'] = player_match_data.apply(lambda row: 
                                                           1 if row.home_away=='away' and row.team_winner==row.team 
                                                             else 0, axis = 1)

player_match_data['team_away_draw'] = player_match_data.apply(lambda row: 
                                                           1 if row.home_away=='away' and row.team_winner=='draw' 
                                                             else 0, axis = 1)

player_match_data['team_away_lose'] = player_match_data.apply(lambda row: 
                                                           1 if row.home_away=='away' and row.team_winner==row.opponent 
                                                             else 0, axis = 1)

In [10]:
player_match_data['winner'] = player_match_data.apply(
    lambda row: 'w' if row.team==row.team_winner else 'd' if row.team_winner=='draw' else 'l', axis = 1)

player_match_data['ht_winner'] = player_match_data.apply(
    lambda row: 'w' if row.team==row.ht_team_winner else 'd' if row.ht_team_winner=='draw' else 'l', axis = 1)


player_match_data['team_win'] = (player_match_data['winner'] == 'w').astype(int)
player_match_data['team_draw'] = (player_match_data['winner'] == 'draw').astype(int)
player_match_data['team_lose'] = (player_match_data['winner'] == 'l').astype(int)

player_match_data['ht_team_win'] = (player_match_data['ht_winner'] == 'w').astype(int)
player_match_data['ht_team_draw'] = (player_match_data['ht_winner'] == 'draw').astype(int)
player_match_data['ht_team_lose'] = (player_match_data['ht_winner'] == 'l').astype(int)

player_match_data.drop('winner', inplace = True, axis = 1)
player_match_data.drop('ht_winner', inplace = True, axis = 1)

In [11]:
player_match_data['team_points'] = player_match_data.apply(
    lambda row: 3 if row.team_win==1 else 0 if row.team_lose==1 else 1, axis = 1)

player_match_data['ht_team_points'] = player_match_data.apply(
    lambda row: 3 if row.ht_team_win==1 else 0 if row.ht_team_lose==1 else 1, axis = 1)

In [12]:
player_match_data['team_xg'] = player_match_data.apply(
    lambda row: row.home_xg if row.team==row.home_team else row.away_xg, axis = 1)

player_match_data['against_xg'] = player_match_data.apply(
    lambda row: row.away_xg if row.team==row.home_team else row.home_xg, axis = 1)

In [13]:
player_match_data['team_scored'] = player_match_data.apply(
    lambda row: row.home_score if row.team==row.home_team else row.away_score, axis = 1)
player_match_data['team_conceded'] = player_match_data.apply(
    lambda row: row.home_score if row.team==row.away_team else row.away_score, axis = 1)

player_match_data['ht_team_scored'] = player_match_data.apply(
    lambda row: row.ht_home_score if row.team==row.home_team else row.ht_away_score, axis = 1)
player_match_data['ht_team_conceded'] = player_match_data.apply(
    lambda row: row.ht_home_score if row.team==row.away_team else row.ht_away_score, axis = 1)

In [14]:
names = player_match_data.name.unique()

player_match_data.position.fillna('No_position', inplace = True)

for nam in tqdm(names, desc = 'Adding unique position to every player'):
        player_pos = []
        player = player_match_data.loc[player_match_data.name==nam, ['date', 'position']]
        player['position'].apply(lambda row: player_pos.extend(row.split(",")))
        pos = max(set(player_pos), key = player_pos.count).strip()
        player_match_data.loc[player_match_data.name==nam, 'main_position'] = pos

Adding unique position to every player: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6267/6267 [06:26<00:00, 16.22it/s]


In [15]:
player_match_data.drop('score', inplace = True, axis = 1)
player_match_data.drop('ht_score', inplace = True, axis = 1)

player_match_data.sort_values('date', inplace = True)

player_match_data.to_csv('analytics/player_match_data.csv', index = False)

### 4. Generate additional tables
#### 4.1. Generate table of all matches with stats for each team (2 columns per match: home and away team)

In [16]:
team_match_columns = ['season', 'dayofweek', 'date', 'start_time', 'home_team', 'home_xg',
       'away_xg', 'away_team', 'attendance', 'venue', 'referee', 'league',
       'gameweek', 'manager', 'team', 'formation', 'home_score', 'away_score',
       'ht_home_score', 'ht_away_score', 'match_home_win', 'match_draw',
       'match_win', 'match_home_lose', 'match_away_lose',
       'ht_match_home_win', 'ht_match_draw', 'ht_match_away_win',
       'ht_match_home_lose', 'ht_match_away_lose', 'opponent',
       'team_winner', 'ht_team_winner', 'home_away', 'team_home_win',
       'team_home_draw', 'team_home_lose', 'team_away_win',
       'team_away_draw', 'team_away_lose', 'team_win', 'team_draw',
       'team_lose', 'ht_team_win', 'ht_team_draw', 'ht_team_lose',
       'team_points', 'ht_team_points', 'team_xg', 'against_xg',
       'team_scored', 'team_conceded', 'ht_team_scored',
       'ht_team_conceded']

redundant_numeric_columns = ['starter', 'minutes', 'x', 'y','shirtnumber']

pct_cols = [x for x in player_match_data.columns if '_pct' in x]

numeric_columns = player_match_data.select_dtypes(include=[np.number]).columns.tolist()
numeric_columns = [col for col in numeric_columns if col not in redundant_numeric_columns]

mean_cols = pct_cols.copy()
mean_cols.append('age')

In [17]:
agg_dict = {col: 'sum' for col in numeric_columns if col not in mean_cols and col not in team_match_columns}

for x in mean_cols:
    agg_dict[x]='mean'
    
team_match_data = player_match_data.groupby(team_match_columns).agg(agg_dict).reset_index()
team_match_data.to_csv('analytics/team_match_data.csv', index = False)

#### 4.2. Generate table of matches

In [18]:
match_columns = ['dayofweek', 'date', 'start_time', 'home_team', 'home_xg',
       'away_xg', 'away_team', 'attendance', 'venue', 'referee', 'league',
       'gameweek', 'season', 'home_score', 'away_score',
       'ht_home_score', 'ht_away_score', 'match_home_win', 'match_draw',
       'match_win', 'match_home_lose', 'match_away_lose',
       'ht_match_home_win', 'ht_match_draw', 'ht_match_away_win',
       'ht_match_home_lose', 'ht_match_away_lose',
       'team_winner', 'ht_team_winner']
redundant_numeric_columns = ['starter', 'minutes', 'x', 'y','shirtnumber', 'opponent', 'team_home_win',
       'team_home_draw', 'team_home_lose', 'team_away_win',
       'team_away_draw', 'team_away_lose', 'team_win', 'team_draw',
       'team_lose', 'ht_team_win', 'ht_team_draw', 'ht_team_lose',
       'team_points', 'ht_team_points']

numeric_columns = team_match_data.select_dtypes(include=[np.number]).columns.tolist()
numeric_columns = [col for col in numeric_columns if col not in redundant_numeric_columns]

mean_cols = pct_cols.copy()
mean_cols.append('age')

In [19]:
agg_dict = {col: 'sum' for col in numeric_columns if col not in mean_cols and col not in match_columns}

for x in mean_cols:
    agg_dict[x]='mean'
    
match_data = team_match_data.groupby(match_columns).agg(agg_dict).reset_index()
match_data.to_csv('analytics/match_data.csv', index = False)

#### 4.3. Generate table of teams per season stats

For example:
- Napoli, 2020-2021,....
- Juventus, 2022-2023,...
- Real Madrid, 2018-2019,...

...

In [20]:
team_columns = ['team', 'league', 'season']

numeric_columns = team_match_data.select_dtypes(include=[np.number]).columns.tolist()
redundant_numeric_columns = ['home_xg','away_xg', 'gameweek', 'x', 'y',
       'shirtnumber', 'minutes', 'home_score', 'away_score',
       'ht_home_score', 'ht_away_score', 'match_home_win', 'match_draw',
       'match_win', 'match_home_lose', 'match_away_lose',
       'ht_match_home_win', 'ht_match_draw', 'ht_match_away_win',
       'ht_match_home_lose', 'ht_match_away_lose']
numeric_columns = [col for col in numeric_columns if col not in redundant_numeric_columns]

mean_cols = pct_cols.copy()
mean_cols.append('age')

In [21]:
leagues = team_match_data.league.unique()
seasons = team_match_data.season.unique()         

agg_dict = {col: 'sum' for col in numeric_columns if col not in mean_cols and col not in team_columns}

for x in mean_cols:
    agg_dict[x]='mean'

grouped_data = team_match_data.groupby(team_columns)
team_data_season = grouped_data.agg(agg_dict).reset_index()
team_data_season['num_matches'] = grouped_data.size().values
team_data_season['managers'] = grouped_data['manager'].agg(lambda x: ', '.join(set(x))).reset_index()['manager']
team_data_season['num_managers'] = grouped_data['manager'].agg(lambda x: len(set(x))).reset_index()['manager']

team_data_season.sort_values('team_points', ascending = False, inplace = True)

team_data_season_avg = team_data_season.copy()

numeric_columns = [col for col in numeric_columns if col not in mean_cols]

team_data_season_avg.loc[:, numeric_columns] = team_data_season_avg.loc[:, numeric_columns].div(team_data_season_avg['num_matches'], axis=0)

team_data_season_avg.sort_values('team_points',ascending = False, inplace = True)

for l in leagues:
    
    for s in seasons:
        
        mask = (team_data_season.league == l) & (team_data_season.season == s)
        
        ls = team_data_season.loc[mask].sort_values('team_points').reset_index(drop = True)
        ls['league_rank'] = range(1,len(ls)+1)
        ls.to_csv(f'analytics/team_data_{l}_{s}.csv', index = False)
        
        lsa = team_data_season_avg.loc[mask].sort_values('team_points').reset_index(drop = True)
        lsa['league_rank'] = range(1,len(lsa)+1)
        lsa.to_csv(f'analytics/team_data_{l}_{s}_avg.csv', index = False)
        
        team_data_season.loc[mask, 'league_rank'] = range(1,len(team_data_season[mask])+1)
        
        team_data_season_avg.loc[mask, 'league_rank'] = range(1,len(team_data_season_avg[mask])+1)

team_data_season_avg.to_csv('analytics/team_data_season_avg.csv', index = False)
team_data_season.to_csv('analytics/team_data_season.csv', index = False)

/tmp/ipykernel_28486/1420723159.py:21: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  team_data_season_avg.loc[:, numeric_columns] = team_data_season_avg.loc[:, numeric_columns].div(team_data_season_avg['num_matches'], axis=0)
/tmp/ipykernel_28486/1420723159.py:41: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  team_data_season_avg.loc[mask, 'league_rank'] = range(1,len(team_data_season_avg[mask])+1)
/tmp/ipykernel_28486/1420723159.py:41: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  team_data_season_avg.loc[mask, 'league_rank'] = range(1,len(team_data_season_avg[mask])+1)


#### 4.4. Generate table of player per season stats

In [22]:
player_columns = ['name', 'main_position', 'team', 'league', 'season']

numeric_columns = player_match_data.select_dtypes(include=[np.number]).columns.tolist()
redundant_numeric_columns = ['home_xg','away_xg', 'gameweek', 'x', 'y',
       'shirtnumber', 'minutes', 'home_score', 'away_score',
       'ht_home_score', 'ht_away_score', 'match_home_win', 'match_draw',
       'match_win', 'match_home_lose', 'match_away_lose',
       'ht_match_home_win', 'ht_match_draw', 'ht_match_away_win',
       'ht_match_home_lose', 'ht_match_away_lose']
numeric_columns = [col for col in numeric_columns if col not in redundant_numeric_columns]

In [23]:
agg_dict = {col: 'sum' for col in numeric_columns if col not in mean_cols and col not in player_columns}
for x in mean_cols:
    agg_dict[x]='mean'

grouped_data = player_match_data.groupby(player_columns)
player_data_season = grouped_data.agg(agg_dict).reset_index()
player_data_season['num_matches'] = grouped_data.size().values

player_data_season.sort_values('team_points', ascending = False)

player_data_season.sort_values('goals', ascending = False, inplace = True)
        
player_data_season.to_csv('analytics/player_data_season.csv', index = False)

player_data_season_avg = player_data_season.copy()

numeric_columns = [col for col in numeric_columns if col not in mean_cols]

player_data_season_avg.loc[:, numeric_columns] = player_data_season_avg.loc[:, numeric_columns].div(player_data_season_avg['num_matches'], axis=0)

player_data_season_avg.sort_values('goals', ascending = False, inplace = True)

player_data_season_avg.to_csv('analytics/player_data_season_avg.csv', index = False)

for l in leagues:
    
    for s in seasons:
        
        ls = player_data_season.loc[
            (player_data_season.league == l) & 
            (player_data_season.season == s)].sort_values('goals').reset_index(drop = True)
        ls.to_csv(f'analytics/player_data_{l}_{s}.csv', index = False)
        
        lsa = player_data_season_avg.loc[
            (player_data_season_avg.league == l) & 
            (player_data_season_avg.season == s)].sort_values('goals').reset_index(drop = True)
        lsa.to_csv(f'analytics/player_data_{l}_{s}_avg.csv', index = False)

/tmp/ipykernel_28486/2089326095.py:19: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  player_data_season_avg.loc[:, numeric_columns] = player_data_season_avg.loc[:, numeric_columns].div(player_data_season_avg['num_matches'], axis=0)


### 5. Conclusion

In this notebook, from 2017-2018 season, next tables were created:

- Table of team statistics for each played match summarized
- Table of each match statistics summarized
- Table of team statistics per each season summarized
- Table of team statistics per each season averaged per match
- Tables of team statistics for each league per each season summarized
- Tables of team statistics for each league per each season averaged per match
- Table of player statistics per season summarized
- Table of player statistics per season averaged per match
- Tables of player statistics for each league per each season summarized
- Tables of player statistics for each league per each season averaged per match

Thank you for your attention! 
